# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,city,cloudiness,country,date_list,humidity,Lat_List,Lng_List,Max_Temp_List,Wind_Speed_List
0,Baykit,36,RU,1623448855,92,61.6700,96.3700,41.76,3.04
1,Ust-Kuyga,96,RU,1623448380,36,70.0167,135.6000,64.40,10.02
2,Lebu,73,CL,1623448342,94,-37.6167,-73.6500,55.20,2.57
3,Kapaa,40,US,1623448328,67,22.0752,-159.3190,82.72,3.00
4,Busselton,82,AU,1623448108,86,-33.6500,115.3333,50.09,5.10
...,...,...,...,...,...,...,...,...,...
579,Shibganj,100,BD,1623449046,90,24.6829,88.1598,79.34,12.55
580,Pringsewu,99,ID,1623449046,96,-5.3582,104.9744,72.28,2.82
581,City of San Pedro,100,PH,1623449046,91,14.3500,121.0167,79.11,3.56
582,Sola,37,VU,1623449047,68,-13.8833,167.5500,79.90,23.24


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat_List", "Lng_List"]]

# Store Humidity in humidity
humidity = weather_data["humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind_Speed_List"] <= 10) & (weather_data["cloudiness"] == 0) & \
                                   (weather_data["Max_Temp_List"] >= 70) & (weather_data["Max_Temp_List"] <= 80)].dropna()

narrowed_city_df

,city,cloudiness,country,date_list,humidity,Lat_List,Lng_List,Max_Temp_List,Wind_Speed_List
39,Cayenne,0,GF,1623448763,90,4.9333,-52.3333,76.08,0.00
85,Ponta do Sol,0,PT,1623448361,49,32.6667,-17.1000,74.03,5.84
156,Marsaxlokk,0,MT,1623448901,88,35.8419,14.5431,70.09,4.61
159,Mahon,0,ES,1623448902,78,39.8885,4.2658,72.16,8.05
332,Kiryat Gat,0,IL,1623448441,63,31.6100,34.7642,74.39,4.74
437,Nador,0,MA,1623448773,88,35.1740,-2.9287,70.05,3.44
457,Marrakesh,0,MA,1623449003,38,31.6315,-8.0083,78.87,4.61
465,Fes,0,MA,1623449005,56,34.0372,-4.9998,73.65,6.91
526,Marand,0,IR,1623449027,59,38.4329,45.7749,70.14,7.31


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["city","country", "Lat_List", "Lng_List"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,city,country,Lat_List,Lng_List,Hotel Name
39,Cayenne,GF,4.9333,-52.3333,
85,Ponta do Sol,PT,32.6667,-17.1000,
156,Marsaxlokk,MT,35.8419,14.5431,
159,Mahon,ES,39.8885,4.2658,
332,Kiryat Gat,IL,31.6100,34.7642,
437,Nador,MA,35.1740,-2.9287,
457,Marrakesh,MA,31.6315,-8.0083,
465,Fes,MA,34.0372,-4.9998,
526,Marand,IR,38.4329,45.7749,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat_List"]
    lng = row["Lng_List"]
    city_name = row["city"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 39: Cayenne.
Closest hotel in Cayenne is Mercure Cayenne Royal Amazonia.
------------
Retrieving Results for Index 85: Ponta do Sol.
Closest hotel in Ponta do Sol is Estalagem Da Ponta Do Sol.
------------
Retrieving Results for Index 156: Marsaxlokk.
Closest hotel in Marsaxlokk is Cugó Gran Macina Grand Harbour.
------------
Retrieving Results for Index 159: Mahon.
Closest hotel in Mahon is Jardí de ses Bruixes Boutique Hotel.
------------
Retrieving Results for Index 332: Kiryat Gat.
Closest hotel in Kiryat Gat is Desert Gat.
------------
Retrieving Results for Index 437: Nador.
Closest hotel in Nador is Hotel Marchica Lagoon Resort, Nador Morocco.
------------
Retrieving Results for Index 457: Marrakesh.
Closest hotel in Marrakesh is Sofitel Marrakech Lounge & Spa.
------------
Retrieving Results for Index 465: Fes.
Closest hotel in Fes is Riad Fes - Relais & Châteaux.
------------
Retrieving Results for Index 526: Marand.
Closest hotel in Marand is Mara

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{city}</dd>
<dt>Country</dt><dd>{country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat_List", "Lng_List"]]

In [11]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))